In [ ]:
# Combine CropGB and LCM, combined codes (note raster calculator won't work via Phython). Example for 2017

#Open the LCM
# Lookup for the values (and not the colour banding)
out_raster = arcpy.sa.Lookup("gb2017lcm25m.tif", "Value"); 
out_raster.save(r"D:\Data\Dropbox\Lactuca\Projects\AgLand SDR\ArcPro_AgLandSDR\ArcPro_AgLandSDR.gdb\Lookup_gb2011")
#Export as LCM2017.tif
arcpy.management.AddField("LCM2017.tif", "BHT", "TEXT", None, None, None, "Broad Habitat Type", "NULLABLE", "NON_REQUIRED", '')
# MANUALLY Reclassify in BHT to standard LCM naming & save file as LCM2017.tif
# Set to colouring correct
arcpy.management.ApplySymbologyFromLayer("LCM2017.tif", "LCM2015.tif", None, "MAINTAIN")

# split off Arable and improved grassland from LCM
out_raster = arcpy.sa.SetNull("LCM2017.tif", "LCM2017.tif", "BHT <> 'Arable and horticulture' And BHT <> 'Improved grassland'"); 
out_raster.save("Arable_Grass")
# Rename MANUALLY to Arable&Grass

# Open the shape cropLayer
#Convert to raster
with arcpy.EnvManager(compression="'LZW' 75", outputCoordinateSystem='PROJCS["British_National_Grid",GEOGCS["GCS_OSGB_1936",DATUM["D_OSGB_1936",SPHEROID["Airy_1830",6377563.396,299.3249646]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",400000.0],PARAMETER["False_Northing",-100000.0],PARAMETER["Central_Meridian",-2.0],PARAMETER["Scale_Factor",0.9996012717],PARAMETER["Latitude_Of_Origin",49.0],UNIT["Meter",1.0]]', snapRaster="LCM2017.tif", cellSize="LCM2017.tif"):
    arcpy.conversion.FeatureToRaster("crops2017_GB", "crop2017", "CropsasRaster2017.tif", "LCM2017.tif")

# Add BHT field
arcpy.management.AddField("CropsasRaster2017.tif", "CropCode", "LONG", None, None, None, "CropCode", "NULLABLE", "NON_REQUIRED", '')
# MANUALLY Reclassify in into standard numbering
# 1 gr, 2 ww, 3 wb, 4 sw, 5 ot, 6 ma, 7 or, 8 sb, 9 po, 10, fb, 11 be
# LookUp and Save to new raster so values match
out_raster = arcpy.sa.Lookup("CropsasRaster2017.tif", "CropCode"); 
out_raster.save(r"D:\Data\Dropbox\Lactuca\Projects\AgLand SDR\ArcPro_AgLandSDR\ArcPro_AgLandSDR.gdb\Lookup_Crops1")
arcpy.management.AddField("Lookup_Crops1", "crops2017", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
# MANUALLY add class names and export as CropsasRaster2017.tif

# set crops and grass over for those areas that the LCM says are arable of improved grass
with arcpy.EnvManager(outputCoordinateSystem='PROJCS["British_National_Grid",GEOGCS["GCS_OSGB_1936",DATUM["D_OSGB_1936",SPHEROID["Airy_1830",6377563.396,299.3249646]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",400000.0],PARAMETER["False_Northing",-100000.0],PARAMETER["Central_Meridian",-2.0],PARAMETER["Scale_Factor",0.9996012717],PARAMETER["Latitude_Of_Origin",49.0],UNIT["Meter",1.0]]', cellSize="Arable&Grass"):
    output_raster = arcpy.sa.RasterCalculator(' Con("Arable&Grass"==3,"CropsasRaster2017.tif", Con("Arable&Grass"==4,"CropsasRaster2017.tif"))'); 
    output_raster.save("MergePhase1")

# set LCM improved grassland to crop improved grass when not yet filled yet    
with arcpy.EnvManager(cellSize="Arable&Grass"):
    output_raster = arcpy.sa.RasterCalculator(' Con(IsNull("MergePhase1")==1, Con("Arable&Grass"==4,1,"MergePhase1"),"MergePhase1")'); 
    output_raster.save("Mergephase2a")
    
# set LCM neutral grass to crop improved grass when set as crop in the crop map
output_raster = arcpy.sa.RasterCalculator(' Con("LCM2017.tif"==5, Con(IsNull("CropsasRaster2017.tif") !=1,1),"Mergephase2a")'); 
output_raster.save("Mergephase2b")

# set LCM calc grass to crop improved grass when set as crop in the crop map
output_raster = arcpy.sa.RasterCalculator(' Con("LCM2017.tif"==6, Con(IsNull("CropsasRaster2017.tif") !=1,1),"Mergephase2b")'); 
output_raster.save("Mergephase2c")

# set LCM acid grass to improved grass when set as crop in the crop map
output_raster = arcpy.sa.RasterCalculator(' Con("LCM2017.tif"==7, Con(IsNull("CropsasRaster2017.tif") !=1,1),"Mergephase2c")'); 
output_raster.save("Mergephase2d")

# set unknown arable from the LCM to "other crops"
output_raster = arcpy.sa.RasterCalculator(' Con( IsNull("Mergephase2d")==1, Con("Arable&Grass"==3,5),"Mergephase2d")'); 
output_raster.save("Mergephase2e")

# Reclassify the crop types to new codings
out_raster = arcpy.sa.Reclassify("Mergephase2e", "Value", "1 401;2 302;3 303;4 304;5 305;6 306;7 307;8 308;9 309;10 310;11 311", "DATA"); 
out_raster.save("Mergephase2f")

# For all other areas that are not set as crops or grass, fill in the lCM types
output_raster = arcpy.sa.RasterCalculator(' Con( IsNull("Mergephase2f") !=1 ,"Mergephase2f","LCM2017.tif")');
output_raster.save("Combine1")

# Add a BHT Field
arcpy.management.AddField("Combine1", "BHT", "TEXT", None, None, None, "Broad Habitat Type", "NULLABLE", "NON_REQUIRED", '')
arcpy.management.ApplySymbologyFromLayer("Combine1", "LCM2015PlusCrops.tif", None, "MAINTAIN")
# MANUALLY Reclassify in BHT the crop map to standard numbering & export as LCM2017PlusCrops.tif 

# Copy symbology over
arcpy.management.ApplySymbologyFromLayer("LCM2017PlusCrops.tif", "Combine1", None, "MAINTAIN")